# Day 5 - Self-Paced Practice: The AI Travel Agent

**Objective:** Reinforce the concept of a tool-using agent by applying it to a new, practical problem: planning a trip.

**Estimated Time:** 45 minutes

**Introduction:**
This optional lab challenges you to build a simple AI Travel Agent. You will give it access to a web search tool and ask it to perform a multi-step research task to create a travel itinerary. This requires the agent to make multiple, distinct calls to its tool to gather all the necessary information before synthesizing a final answer.

## 1. Setup

We will set up our environment and the LangChain agent components, just as we did in the core lab. This includes installing the necessary libraries and initializing our LLM client.

In [ ]:
import sys
import os

# Add the project's root directory to the Python path
try:
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
except IndexError:
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

# This helper will install packages if they are not found
import importlib
def install_if_missing(package):
    try:
        importlib.import_module(package)
    except ImportError:
        print(f"{package} not found, installing...")
        import subprocess
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])

install_if_missing('langchain')
install_if_missing('langchain_community')
install_if_missing('langchain_openai')
install_if_missing('tavily')

from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from utils import setup_ll_m_client

# For agentic tasks, a powerful model is recommended.
client, model_name, api_provider = setup_llm_client(model_name="gpt-4o")
llm = ChatOpenAI(model=model_name)

## 2. The Problem Statement

Your task is to build an agent that can plan a 3-day trip to Paris, France. The agent needs to be able to research flights, activities, and restaurants.

### ⭐ Deeper Dive: The ReAct Framework (Reasoning and Acting)

How does an agent actually "use" a tool? Most modern agents follow a pattern called **ReAct**, which stands for **Reason-Act**.

When you give the agent a complex task, it enters a loop:

1.  **Reason (Thought):** The agent first thinks about the overall goal and its available tools. It breaks the problem down and decides on a single, concrete action it can take *right now*. For example: `"The user needs flight options. I should use the search tool to find flights from JFK to CDG."`
2.  **Act:** The agent executes the chosen action. This means calling the tool (e.g., `TavilySearchResults(query='flights from JFK to CDG')`).
3.  **Observe:** The agent gets the result (the "observation") back from the tool (e.g., a list of flight search results).
4.  **Repeat:** The agent takes this new information, re-evaluates the overall goal, and enters the **Reason** step again. `"Okay, I have the flights. Now the user needs tourist activities. I should use the search tool again to find activities in Paris."`

This **Thought -> Action -> Observation** loop continues until the agent has gathered enough information to formulate a final answer. When you set `verbose=True` in a LangChain `AgentExecutor`, you are watching this ReAct loop happen in real-time!

## 3. Your Task

Build a LangChain agent that uses the Tavily Search tool to create a complete travel itinerary.

#### 💡 Pro-Tip: Prompting for Multi-Step Tasks

When you need an agent to perform several distinct tasks (like finding flights, activities, *and* restaurants), it's best to list them all clearly in the initial prompt. This allows the agent to form a better high-level plan from the start.

Instead of asking questions one by one, bundle them into a single, comprehensive request. This encourages the agent to complete its full research loop before giving you the final, synthesized answer.

In [ ]:
# 1. Instantiate the tool. We'll give it a clear name for the agent.
search_tool = TavilySearchResults(max_results=3)
search_tool.name = "web_search"
tools = [search_tool]

# 2. Create the prompt template. The persona helps guide its tone.
travel_agent_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful travel agent. You are thorough and provide detailed, well-organized itineraries. You must find all the information requested by the user before giving your final answer."),
    ("user", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

# 3. Create the agent and executor
travel_agent = create_tool_calling_agent(llm, tools, travel_agent_prompt)
travel_agent_executor = AgentExecutor(agent=travel_agent, tools=tools, verbose=True)

# 4. Define the detailed, multi-step query
trip_query = """
Please create a 3-day travel itinerary for a trip to Paris, France, from New York City.

Specifically, I need you to research and include the following:
1.  Two different round-trip flight options from JFK to CDG.
2.  One unique tourist activity for each of the three days.
3.  One highly-rated restaurant recommendation near the Eiffel Tower.

Please synthesize all of this information into a single, final itinerary.
"""

# 5. Invoke the agent
print("--- Planning Trip to Paris ---")
trip_result = travel_agent_executor.invoke({"input": trip_query})

print("\n--- Final Itinerary ---")
print(trip_result['output'])

## Lab Conclusion

Well done! You have successfully built a practical, tool-using agent that can perform a multi-step research task. This lab reinforces how agents can break down a complex query into smaller, manageable steps, use their tools to gather information for each step, and then synthesize the results into a comprehensive final answer. You also learned about the underlying ReAct framework that powers this agentic behavior.